In [1]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph
from sklearn.manifold import TSNE
# from fastText import tokenize, load_model
import gensim
import glob
import string

In [2]:
df1 = pd.read_csv("grantnav-20180526072920.csv",encoding='utf-8')
df1 = df1[['Title', 'Description']]
df1 = df1.dropna()

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,10,13,19,20,21,23,24,25,31,34,35,36,37,38,39,40,41,42,43,44,45,46,47,49,50,51,53,54,55,57,58,59,61,62,63,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
descs = set(df1[~df1['Description'].isin(['Placeholder', 'None', 'donation','Ynot!', 'A', 'a', '-', 'none', '004649'])].Description.values)
# Removing some garbage values

In [4]:
titles = set(df1[~df1['Title'].isin(['.','C','-','x',])].Title.values)

Descriptions should be somewhat cleaned now in terms of entries and values. 

In [5]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    normalized = " ".join(lemma.lemmatize(word,'v') for word in stop_free.split())
    x = normalized.split()
    y = [s for s in x if len(s) > 2]
    return y

In [9]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/paperspace/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [11]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
doc_clean = [clean(doc) for doc in descs]

In [12]:
from gensim import corpora
# Creating term dictionary of corpus, where each unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)
# Filter terms which occurs in less than 4 articles & more than 40% of the articles 
dictionary.filter_extremes(no_below=4, no_above=0.4)
filter_worlds = ['grant', 'award', ]

In [13]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [17]:
# from gensim.models.ldamodel import LdaModel as Lda
from gensim import models
# Creating the object for LDA model using gensim library & Training LDA model on the document term matrix.
lda = models.LdaMulticore(doc_term_matrix, id2word=dictionary, num_topics=50, workers=3)
# ldamodel = Lda(doc_term_matrix, num_topics=50, id2word = dictionary, passes=50, iterations=500)

In [19]:
for i,topic in enumerate(lda.print_topics(num_topics=50, num_words=10)):
   words = topic[1].split("+")
   print(words,"\n")

['0.031*"award" ', ' 0.025*"anniversary" ', ' 0.022*"members" ', ' 0.021*"hold" ', ' 0.021*"end" ', ' 0.017*"use" ', ' 0.016*"60th" ', ' 0.016*"group" ', ' 0.015*"commemorate" ', ' 0.014*"event"'] 

['0.031*"radio" ', ' 0.017*"hospital" ', ' 0.015*"new" ', ' 0.011*"station" ', ' 0.011*"cancer" ', ' 0.010*"broadcast" ', ' 0.010*"rehabilitation" ', ' 0.009*"patients" ', ' 0.008*"breast" ', ' 0.007*"library"'] 

['0.064*"young" ', ' 0.054*"people" ', ' 0.032*"provide" ', ' 0.030*"activities" ', ' 0.024*"use" ', ' 0.021*"fund" ', ' 0.021*"group" ', ' 0.017*"skills" ', ' 0.016*"social" ', ' 0.015*"improve"'] 

['0.023*"bowl" ', ' 0.019*"bike" ', ' 0.016*"water" ', ' 0.014*"mat" ', ' 0.009*"supply" ', ' 0.009*"would" ', ' 0.009*"power" ', ' 0.009*"lighting," ', ' 0.009*"restoration" ', ' 0.008*"conservation"'] 

['0.013*"replace" ', ' 0.012*"use" ', ' 0.009*"safety" ', ' 0.007*"old" ', ' 0.006*"maternal" ', ' 0.006*"months" ', ' 0.006*"health" ', ' 0.006*"intervention" ', ' 0.006*"malaria" '

In [24]:
import pickle
ldafile = open('multicore simple lda','wb')
pickle.dump(lda,ldafile)
ldafile.close()